In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
test_data=pd.read_csv('test.csv')

In [ ]:
X_data_submit=test_data.values
X_data_submit=np.reshape(X_data_submit,[-1,28,28,1])
del test_data

In [ ]:
q1=tf.FIFOQueue(capacity=800,dtypes=tf.float32,shapes=[28,28,1])

In [ ]:
enqueue_op1= q1.enqueue_many(vals=X_data_submit)

In [ ]:
numberOfThreads = 3
qr1 = tf.train.QueueRunner(q1, [enqueue_op1] * numberOfThreads)
tf.train.add_queue_runner(qr1)

In [7]:
#now make a computation graph
def ConvLayer(X,no_of_filters,filter_size):
    in_channels=int(X.shape[3])
    W=tf.get_variable("filter",dtype=tf.float32,initializer=tf.random_normal([filter_size,filter_size,in_channels,no_of_filters]))
    b=tf.get_variable("bias",dtype=tf.float32,initializer=tf.random_normal([no_of_filters]))
    
    out=tf.nn.relu(tf.add(tf.nn.conv2d(input=X,filter=W,strides=[1,1,1,1],padding='SAME'),b))
    pool=tf.nn.max_pool(value=out,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    return pool

In [8]:
def FcLayer(X,out_layer):
    in_layer=int(X.shape[1])
    
    W=tf.get_variable("Weights",dtype=tf.float32,initializer=tf.random_normal([in_layer,out_layer],dtype=tf.float32))
    b=tf.get_variable("Bias",dtype=tf.float32,initializer=tf.random_normal([out_layer],dtype=tf.float32))
    
    out=tf.add(tf.matmul(X,W),b)
    return out

In [9]:
batch_size=200
X=q1.dequeue_many(batch_size)

In [10]:
with tf.variable_scope("Conv_layer1"):
    out1=ConvLayer(X,32,5)
    out1=tf.nn.dropout(out1,1.0)
with tf.variable_scope("Conv_layer2"):
    out2=ConvLayer(out1,64,5)
    out2=tf.nn.dropout(out2,1.0)
with tf.variable_scope("Conv_layer3"):
    out3=ConvLayer(out2,128,2)
    out3=tf.nn.dropout(out3,1.0)

channels=int(out3.shape[3])
length=int(out3.shape[2])
breadth=int(out3.shape[1])

out3=tf.reshape(out3,[-1,length*breadth*channels])

with tf.variable_scope("Fc_layer1"):
    fc_out1=FcLayer(out3,3072)
    fc_out1=tf.nn.relu(fc_out1)
    fc_out1=tf.nn.dropout(fc_out1,1.0)

with tf.variable_scope("Fc_layer2"):
    fc_out2=FcLayer(fc_out1,10)

prediction=tf.arg_max(fc_out2,dimension=1)

In [11]:
sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

In [12]:
tf.train.Saver().restore(sess,'trained_params/')

In [13]:
no_of_batches=int(X_data_submit.shape[0]/batch_size)

In [ ]:
predicted=np.zeros([X_data_submit.shape[0]])
for _ in range(no_of_batches):
    predicted[_*batch_size:(_+1)*batch_size]=sess.run(prediction)

predicted=predicted.astype(copy=False,dtype=np.int32)

In [ ]:
ImageId=np.zeros([X_data_submit.shape[0]],dtype=np.int32)
for _ in range(0,X_data_submit.shape[0]):
    ImageId[_]=_+1

In [ ]:
Dataframe=pd.DataFrame(data={'ImageId':ImageId,'Label':predicted})
Dataframe.to_csv('submission.csv',index=False)

In [ ]:
coord.request_stop()
coord.join(threads)

In [13]:
%qtconsole